In [1]:
from dp import read_trees, tree_level, print_tree
from utils import Statistics, merge_statistics

gold_trees = read_trees("/efs/dengcai/BCR/dp/out/gold")
tot = len(gold_trees)

In [2]:
#BCR
def read_scores(fname):
    return [ float(x.strip()) for x in open(fname).readlines()]
NUM=10
dropout=0.15
old="123"
new="1"
pred_trees, pred_scores = [], []
for sample_seed in range(1, NUM+1):
    pred_trees = pred_trees + read_trees(f"/efs/dengcai/BCR/dp/out/stackptr{new}.dropout{dropout}sample{sample_seed}.pred")
    pred_scores = pred_scores + read_scores(f"/efs/dengcai/BCR/dp/out/stackptr{new}.dropout{dropout}sample{sample_seed}.deepbiaf{old}.score")
assert len(pred_trees) == len(pred_scores), (len(pred_trees), len(pred_scores))
final_pred_trees = []
for i in range(tot):
    pred_trees_i = pred_trees[i::tot]
    pred_scores_i = pred_scores[i::tot]
    idx = list(range(NUM))
    assert len(idx) == len(pred_trees_i) == len(pred_scores_i)
    idx.sort(key=lambda x:pred_scores_i[x])
    picked = idx[0]
    final_pred_trees.append(pred_trees_i[picked])
BCR_trees = final_pred_trees


#distillation
m1 = "deepbiaf"
m2 = "stackptr"
seed=123
distillation_trees = read_trees(f"/efs/dengcai/BCR/dp/out/gkd.{m2}.{m1}{seed}.1.pred")


#untreated
m2 = "stackptr"
s2 = "1"
untreated_trees = read_trees(f"/efs/dengcai/BCR/dp/out/{m2}{s2}.pred")


#ensemble
m1 = "deepbiaf"
m2 = "stackptr"
s = "A"
ensemble_trees = read_trees(f"/efs/dengcai/BCR/dp/out/{m2}{s2}.pred")

#old
m2 = "deepbiaf"
s2 = "123"
old_trees = read_trees(f"/efs/dengcai/BCR/dp/out/{m2}{s2}.pred")

hold_all = {}
for name, pred_trees in zip(['BCR', 'distill', 'untreated', 'ensemble'], [BCR_trees, distillation_trees, untreated_trees, ensemble_trees]):
    hold_all[name] = tree_level(gold_trees, old_trees, pred_trees, scheme='unlabeled')
    

In [3]:
cnt = 0
for idx, (bcr, distill, untreated, ensemble) in enumerate(zip(hold_all['BCR'], hold_all['distill'], hold_all['untreated'], hold_all['ensemble'])):
    if 'word' not in bcr.keys:
        continue
        
    if 'word' not in distill.keys:
        continue
    
    if 'word' not in untreated.keys:
        continue
        
    if 'word' not in ensemble.keys:
        continue
        
    cnt += 1
    if bcr['word'].nfi < distill['word'].nfi and  \
    bcr['word'].nfi < untreated['word'].nfi and \
    bcr['word'].nfi < ensemble['word'].nfi:
        print(print_tree(BCR_trees[idx]["root"], BCR_trees[idx]["tree"]))
print (cnt)       

( ( ( ) ( And ) ( , ) ( ( by ) ( the ) way ) ( , ) ( is ) ( anybody ( else ) ) ( just ) ( ( a ) little ) nostalgic ( ( for ) ( the ) days ) ( ( when ) ( that ) ( was ) ( a ) ( good ) thing ) ( ? ) ( ) ) )
( ( I ) ( 'm ) ( not ) <_UNK> ( ( of ) ( the ) ( ( ( Google ) ( - ) hates ) ( - ) privacy ) argument ) )
( ( ( The ) President ) ( has ) ( also ) said ( ( he ) ( would ) like ( ( to ) see ( Israel ) ( wiped ( ( off ) ( the ) map ( ( which ) ( he ) ( could ) ( n't ) ( even ) begin ( ( to ) try ( ( without ) ( nuclear ) weapons ) ) ) ) ) ) ) ( . ) )
( ( ( Important ) news ( ( such ( as ) ) ( President ( Bush ) ( 's ) ) ( miniscule ) <_UNK> ( ( on ) ( his ) ( marriage ) amendment ( ( / ) anti-gay ) ( ( ( ) update ( : ) ( ( he ) ( 's ) ( not ) ( against ) gays ( ( in ) ( the ) bedroom ) ( , ) ( ( just ) ( at ) ( the ) <_UNK> ( , ) ( ( where ) ( of ( course ) ) ( ( their ) relationships ) ( should ) ( not ) ( only ) ( be ) frowned ( upon ) ( ( but ) ( should ) ( be ) ( constitutionally ) <

In [4]:
def breakdown(x, prefix, max_rows=None):
    keys = [k for k in x.keys if k.startswith(prefix)]
    print (f"{prefix.split('#')[0]}#NRF#PFR#old ACC#new ACC#NFI#Count")
    for k in sorted(keys, key=lambda k:-x[k].nfr)[:max_rows]:
        xxx = k.split('#')[-1]
        x[k].report_table(f"{xxx}")
    print ("="*55)
#which = 'deepbiaf.labeled'
#which = 'stackptr.labeled'
#which = 'deepbiaf.stackptr.labeled'
which = 'stackptr.deepbiaf.unlabeled'

stat = hold_all[which]

for prefix in ["pos#", "dis#", "headpos", "rel", "size#", "depth#", "rootpos#", "linearization#"]:
    if prefix == "linearization#":
        breakdown(stat, prefix, 10)
    else:
        breakdown(stat, prefix)

KeyError: 'stackptr.deepbiaf.unlabeled'

In [ ]:
#which='deepbiaf.unlabeled'
#which='stackptr.unlabeled'
#which='deepbiaf.stackptr.unlabeled'
which='stackptr.deepbiaf.unlabeled'
stat = hold_all[which]
def pair_analysis(x, prefix):
    keys = [k for k in x.keys if k.startswith(prefix)]
    print (f"{prefix.split('#')[0]}#both/any#both#Only#Both NO#Count")
    for k in sorted(keys, key=lambda k:-x[k].yy/(x[k].any+1e-9)):
        xxx = k.split('#')[-1]
        print (f"{xxx}#{x[k].yy/(x[k].any+1e-9)*100:.2f}%#{x[k].yy}#{x[k].ny+x[k].yn}#{x[k].nn}#{x[k].tot}")
    print ("="*55)
pair_analysis(stat, "pair_token_seqdis#")
pair_analysis(stat, "pair_token_dep#")
pair_analysis(stat, "pair_token_treedis#")
pair_analysis(stat, "pair_subtree_treedis#")